In [68]:
pip install scikeras

In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from google.colab import drive
from keras.models import Sequential
from sklearn.compose import ColumnTransformer
from keras.layers import Dense,Dropout,BatchNormalization
from keras.utils import to_categorical,plot_model
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection  import cross_val_score,train_test_split,cross_val_predict
from keras.metrics import categorical_accuracy,Precision, Recall,F1Score
from sklearn.metrics import confusion_matrix
import pandas as pd
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
import numpy as np


In [70]:

# Montar o Google Drive
drive.mount('/content/drive')
caminho_arquivo = '/content/drive/MyDrive/Colab Notebooks/estudante/student-por.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:

dados = pd.read_csv(caminho_arquivo, sep=";")
dados.iloc[1:10, :]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,6,12,12,13
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,13,12,13
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,2,10,13,13
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,15,16,17
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,12,12,13


In [72]:
dados = dados.drop('nursery', axis=1)


In [73]:
valores = {
    'school': 'GP',
    'sex': 'F',
    'age': 0,
    'address': 'U',
    'famsize': 'GT3',
    'Pstatus': 'T',
    'Medu': 0,
    'Fedu': 0,
    'Mjob': 'other',
    'Fjob': 'other',
    'reason': 'other',
    'guardian': 'mother',
    'traveltime': 0,
    'studytime': 0,
    'failures': 0,
    'schoolsup': 'no',
    'famsup': 'no',
    'paid': 'no',
    'activities': 'no',
    'nursery': 'no',
    'higher': 'no',
    'internet': 'no',
    'romantic': 'no',
    'famrel': 0,
    'freetime': 0,
    'goout': 0,
    'Dalc': 0,
    'Walc': 0,
    'health': 0,
    'absences': 0,
    'G1': 0,
    'G2': 0,
    'G3': 0
}
dados= dados.fillna(value=valores)
print(dados.head())

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        4   0  11  11  
1      5        3      3     1     1      3        2   9  11  11  
2      4        3      2     2     3      3        6  12  13  12  
3      3        2      2     1     1      5        0  14  14  14  
4      4        3      2     1     2      5        0  11  13  13  

[5 rows x 32 columns]


In [74]:
print(dados.loc[1:5,:])

  school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
1     GP   F   17       U     GT3       T     1     1   at_home     other   
2     GP   F   15       U     LE3       T     1     1   at_home     other   
3     GP   F   15       U     GT3       T     4     2    health  services   
4     GP   F   16       U     GT3       T     3     3     other     other   
5     GP   M   16       U     LE3       T     4     3  services     other   

   ... famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
1  ...      5        3      3     1     1      3        2   9  11  11  
2  ...      4        3      2     2     3      3        6  12  13  12  
3  ...      3        2      2     1     1      5        0  14  14  14  
4  ...      4        3      2     1     2      5        0  11  13  13  
5  ...      5        4      2     1     2      5        6  12  12  13  

[5 rows x 32 columns]


In [75]:
nota_final = dados['G3'].values
atributos = dados.drop(columns=['G3']).values


In [76]:
nota_final

array([11, 11, 12, 14, 13, 13, 13, 13, 17, 13, 14, 13, 12, 13, 15, 17, 14,
       14,  7, 12, 14, 12, 14, 10, 10, 12, 12, 11, 13, 12, 11, 15, 15, 12,
       12, 11, 14, 13, 12, 12, 10, 11, 15, 10, 11, 11, 13, 17, 13, 12, 13,
       16,  9, 12, 13, 12, 15, 16, 14, 16, 16, 16, 10, 13, 12, 16, 12, 10,
       11, 15, 11, 10, 11, 14, 11, 11, 11, 13, 10, 11, 12,  9, 11, 13, 12,
       12, 11, 15, 11, 10, 11, 13, 12, 14, 12, 13, 11, 12, 13, 13,  8, 16,
       12, 10, 16, 10, 10, 14, 11, 14, 14, 11, 10, 18, 10, 14, 16, 15, 11,
       14, 14, 13, 13, 13, 11,  9, 11, 11, 15, 13, 12,  8, 11, 13, 12, 14,
       11, 11, 11, 15, 10, 13, 12, 11, 11, 10, 10, 14,  9, 11,  9, 13, 11,
       13, 11,  6, 12, 10, 11, 13, 11,  8, 11,  0, 10, 13, 11, 13,  8, 10,
       11, 11,  1, 10,  9,  8, 10,  8,  8,  8, 11, 18, 13, 17, 10, 18, 10,
       13, 15, 11, 14, 10, 11, 13, 11, 13, 17, 14, 16, 14, 11, 16, 14, 10,
       13, 12, 12, 10, 12, 16, 14, 12, 16, 11, 15, 12, 15, 13, 13,  8, 12,
       15, 13, 12, 12, 12

In [77]:
atributos = np.nan_to_num(dados)

In [81]:
# Identificar colunas com tipo de dado 'object' (não numérico)
colunas_objeto = dados.select_dtypes(include=['object']).columns

# Aplicar a codificação de etiquetas nessas colunas
labelencoder = LabelEncoder()
for coluna in colunas_objeto:
    atributos[coluna] = labelencoder.fit_transform(atributos[coluna])




IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
atributos[0:8,:]

In [ ]:
#one hot enconding

onehotencoder = ColumnTransformer([('one_hot_encoder', OneHotEncoder(), [8,9,10,11])],
                                  remainder='passthrough')

atributos = onehotencoder.fit_transform(atributos)

In [ ]:
atributos[0:5,:]

In [ ]:
atributos.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


# Criação do modelo
regressor = Sequential()
regressor.add(Dense(units=25, activation='relu', input_dim=50))  # Ajuste o número de unidades e a dimensão de entrada para corresponder ao número de atributos
regressor.add(Dense(units=25, activation='relu'))
regressor.add(Dense(units=1, activation='linear'))
regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error', 'mae', 'mse', 'mape', 'msle'])

#metricas adicionada:
#MAPE avalia a precisão de um modelo em fazer previsões
# MSLE metrica de erro usada para problemas de regressão onde a saída pode ter uma escala muito ampla

In [ ]:
print("Tipo de dado dos atributos:", atributos.dtype)
print("Tipo de dado dos preços reais:", nota_final.dtype)
print("Forma dos atributos:", atributos.shape)
print("Forma dos preços reais:", nota_final.shape)
print("Tipo de dado dos atributos:", atributos.dtype)
print("Tipo de dado dos preços reais:", nota_final.dtype)
print("Número de amostras nos atributos:", len(atributos))
print("Número de amostras nos preços reais:", len(nota_final))


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjunto de treinamento e validação
atributos_treino, atributos_validacao, preco_real_treino, preco_real_validacao = train_test_split(atributos, nota_final, test_size=0.2, random_state=42)

# Treinar o modelo
history = regressor.fit(atributos_treino, preco_real_treino, batch_size=200, epochs=100, validation_data=(atributos_validacao, preco_real_validacao))

# Plotar as curvas de aprendizado
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Loss (Treinamento)')
plt.plot(history.history['val_loss'], label='Loss (Validação)')
plt.title('Curvas de Aprendizado')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calcular previsões usando o conjunto de validação
previsoes_validacao = regressor.predict(atributos_validacao).ravel()

# Calcular métricas
#média das diferenças absolutas entre as previsões do modelo e os valores reais.
mae = mean_absolute_error(preco_real_validacao, previsoes_validacao)
# média das diferenças quadráticas entre as previsões do modelo e os valores reais.
mse = mean_squared_error(preco_real_validacao, previsoes_validacao)
# medida estatística que indica a proporção da variabilidade dos valores de resposta que é explicada pelo modelo.
r2 = r2_score(preco_real_validacao, previsoes_validacao)

print("Erro Absoluto Médio (MAE):", mae)
print("Erro Quadrático Médio (MSE):", mse)
print("Coeficiente de Determinação (R²):", r2)

In [ ]:
previsoes = regressor.predict(atributos).ravel()

In [ ]:
print("Previsões:")
print(previsoes)
print("Valores reais:")
print(preco_real)

In [ ]:
print(preco_real[0:100]-previsoes[0:100])

In [ ]:
plt.figure(figsize=(20, 6))
plt.scatter(range(649), preco_real[0:649], color='blue', label='Valores Reais')
plt.scatter(range(649), previsoes[0:649], color='red', label='Previsões')
plt.xlabel('Índice da Amostra')
plt.ylabel('Valor')
plt.title('Comparação entre Valores Reais e Previsões')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Criar um array com o número de amostras para o eixo x
amostras = range(649)

# Plotar previsões vs valores reais como linhas conectadas
plt.figure(figsize=(20, 6))
plt.plot(amostras, preco_real[0:649], color='blue', label='Valores Reais')
plt.plot(amostras, previsoes[0:649], color='red', label='Previsões')
plt.xlabel('Índice da Amostra')
plt.ylabel('Valor')
plt.title('Comparação entre Valores Reais e Previsões')
plt.legend()
plt.grid(True)
plt.show()
